In [1]:
from nuq import NuqClassifier

import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
def generate_points_on_circle(circle_r=10, circle_x=0, circle_y=0, n_points=10, uniform=True):
    if not uniform:
        alpha = 2 * np.pi * np.random.rand(n_points)
    else:
        alpha = 2 * np.pi * np.linspace(start=0, stop=1, num=n_points)

    x = circle_r * np.cos(alpha) + circle_x
    y = circle_r * np.sin(alpha) + circle_y

    return np.vstack((x, y)).T

In [3]:
X = generate_points_on_circle(n_points=100)

In [4]:
import matplotlib.pyplot as plt
%matplotlib widget

In [5]:
plt.close()
plt.figure()

plt.scatter(X[:, 0], X[:, 1])
plt.scatter(0, 0, color='red')
plt.axis('equal')
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
nuq = NuqClassifier(
    method='all_data',
    tune_bandwidth=False,
    bandwidth=np.array([1., 1.])
)

In [7]:
y = np.zeros(X.shape[0], dtype='int')

In [8]:
nuq.fit(X, y)

NuqClassifier(bandwidth=array([1., 1.]), method='all_data', n_neighbors=100,
              tune_bandwidth=False)

In [9]:
nuq.predict_proba(np.zeros((1, 2)))

{'probs': array([[0.]]), 'probsm1': array([[-inf]])}

In [20]:
uncertainties = nuq.predict_uncertainty(np.zeros((1, 2)), )

> /home/nkotelevskii/github/nw_uncertainty/nuq/method/aux_functions.py(145)log_asymptotic_var()
    143     # log_denominator_safe = logsumexp(np.concatenate([log_denominator[None], -40 * np.ones(broadcast_shape)], axis=0),
    144     #                                  axis=0)
--> 145     return log_numerator - log_denominator  # - log_denominator_safe
    146 
    147 



ipdb>  n


--Return--
array([[32.26981864]])
> /home/nkotelevskii/github/nw_uncertainty/nuq/method/aux_functions.py(145)log_asymptotic_var()
    143     # log_denominator_safe = logsumexp(np.concatenate([log_denominator[None], -40 * np.ones(broadcast_shape)], axis=0),
    144     #                                  axis=0)
--> 145     return log_numerator - log_denominator  # - log_denominator_safe
    146 
    147 



ipdb>  n


> /home/nkotelevskii/github/nw_uncertainty/nuq/method/nuq.py(141)predict_uncertainty()
    139                                                 bandwidth=self.bandwidth,
    140                                                 n=self.n_neighbors, dim=self.training_embeddings_.shape[1])
--> 141                 Ue = log_half_gaussian_mean(asymptotic_var=log_as_var).squeeze()
    142                 Ua = np.min(f1_hat_y_x, axis=1, keepdims=True)
    143                 Ua = logsumexp(



...

array([[32.26981864]])


ipdb>  log_as_var[0, 0]


32.26981863527706


ipdb>  q


BdbQuit: 

In [11]:
uncertainties

{'epistemic': array(16.60226515),
 'aleatoric': -11.512925464970229,
 'total': 16.602265145554362}

In [12]:
nuq.get_kde(np.zeros((1, 2)))

array([[-50.91893853]])

In [13]:
C = np.log(2.) - 0.75 * np.log(np.pi) - 0.5 * np.log(X.shape[0]) - 0.5 * np.sum(np.log(nuq.bandwidth))

In [14]:
C

-2.4679853268211507

In [15]:
0.5 * np.log(nuq.coeff) - 0.5 * nuq.get_kde(np.zeros((1, 2))) + C

array([[17.23502121]])

In [16]:
17.23502121 - 18.90485024

-1.669829029999999

In [17]:
print("KDE error:", -0.5 * np.log(2 * np.pi) - 50 - nuq.get_kde(np.zeros((1, 2))))

KDE error: [[0.]]


In [18]:
print("Ue error:", 1.25 * np.log(2.) - 0.5 * np.log(np.pi) - 0.5 * np.log(X.shape[0]) + 0.5 * np.log(nuq.coeff) + 25 - uncertainties['epistemic'])

Ue error: 0.6327560617423238
